In [20]:
import pandas as pd

def read_investment_log(file_path):
    investments = []
    
    with open(file_path, 'r', encoding='utf-8') as file:
        for line in file:
            parts = line.strip().split(', ')
            id_part = parts[0].split(': ')[1]
            target_part = parts[1].split(': ')[1]
            amount_part = int(parts[2].split(': ')[1])
            investments.append({
                '투자자': id_part,
                '투자 대상': target_part,
                '투자 금액': amount_part
            })
    
    return investments

def aggregate_investments(investments):
    df = pd.DataFrame(investments)
    
    # 투자 대상별로 총 투자 금액을 구함
    total_investment = df.groupby('투자 대상')['투자 금액'].sum().reset_index().rename(columns={'투자 금액': '총 투자 금액'})
    
    # 각 투자 대상별로 투자자와 투자 금액을 문자열로 연결
    investor_details = df.groupby(['투자 대상', '투자자'])['투자 금액'].sum().reset_index()
    investor_details['투자 내역'] = investor_details.apply(lambda x: f"{x['투자자']} : {x['투자 금액']}", axis=1)
    
    # 투자 대상별로 투자 내역을 하나의 문자열로 결합
    detailed_investment = investor_details.groupby('투자 대상')['투자 내역'].apply(', '.join).reset_index()
    
    # 총 투자 금액 데이터프레임과 투자 내역 데이터프레임을 병합
    result = pd.merge(total_investment, detailed_investment, on='투자 대상')
    
    return result

# 사용 예시
file_path = 'investment_log.txt'
investments = read_investment_log(file_path)
aggregated_investments = aggregate_investments(investments)

# 결과 출력
print(aggregated_investments)


  투자 대상  총 투자 금액                                              투자 내역
0   구도윤     1600  구도윤 : 400, 김강현 : 300, 김대성 : 100, 임도원 : 100, 전해...
1   권태희      700                               구도윤 : 300, 임도원 : 400
2   김강현     2050  권태희 : 500, 김동우 : 600, 이도형 : 400, 전해성 : 250, 홍길...
3   김대성      950                               김강현 : 700, 전해성 : 250
4   서진영      800                    김동우 : 200, 이도형 : 300, 전해성 : 300
5   이도형      700         구도윤 : 300, 권태희 : 100, 김대성 : 100, 임도원 : 200
6   임도원     1000         권태희 : 100, 김대성 : 100, 이도형 : 300, 홍길동 : 500
7   전해성     2000         김대성 : 700, 김동우 : 200, 임도원 : 300, 홍길동 : 800
8   한민우      300                                          권태희 : 300
